___
___
___
# CLASSIFICATION
___
___
___

In [37]:
import pandas as pd

df = pd.read_csv('../dataset.csv')

# on va faire du LLM avec python pandas scikit-learn
# voici mon contexte : j'ai récupérer des évènements (artisitques, etc.) sur internet. j'ai un csv avec comme features : url	title	description. De plus j'ai trois target : cat1	cat2	cat3
# je voudrai faire de la catégorisation à partir de ces csv. Je ne suis pas sûr que l'URL soit totalement pertinent. Peut être faut-il récupérer le nom de domaine.

___
### 1. Préparation des données
___
tout d'abord, vous devez charger les données à partir du fichier CSV en utilisant Pandas. Ensuite, vous pouvez extraire le nom de domaine à partir de l'URL en utilisant la bibliothèque tldextract. Vous pouvez ensuite ajouter une nouvelle colonne au DataFrame Pandas contenant le nom de domaine.

In [38]:
def extract_domain(url):
    url = url[url.find('//')+2:]
    url = url[:url.find('/')]
    return url

df['domain'] = df['url'].apply(extract_domain)
df[['domain', 'url']].head(1)

,domain,url
0,www.tourisme-cambresis.fr,https://www.tourisme-cambresis.fr/1-les-templi...


___
### 2. Prétraitement des données
___
vous devez ensuite prétraiter les données textuelles dans les colonnes title et description. Vous pouvez par exemple supprimer les ponctuations, les chiffres et les caractères spéciaux, convertir les textes en minuscules, supprimer les stop-words et effectuer une stemming ou une lemmatization. Vous pouvez utiliser la bibliothèque nltk pour effectuer ces opérations.

In [39]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download('stopwords')
nltk.download('punkt')

def preprocess_text(text):
    # Vérifier que la valeur d'entrée est une chaîne de caractères
    if not isinstance(text, str):
        return ''
    
    # Tokenization
    tokens = nltk.word_tokenize(text)

    # Supprimer les ponctuations, les chiffres et les caractères spéciaux
    tokens = [token for token in tokens if token.isalpha()]

    # Convertir en minuscules
    tokens = [token.lower() for token in tokens]

    # Supprimer les stop-words
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]

    # Reconstruction du texte prétraité
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

display(df.head(1))
df['title'] = df['title'].apply(preprocess_text)
df['description'] = df['description'].apply(preprocess_text)
display(df.head(1))


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/utilisateur/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/utilisateur/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Unnamed: 0,url,title,description,cat1,cat2,cat3,domain
0,0,https://www.tourisme-cambresis.fr/1-les-templi...,"Aventure-jeu : ""Les Templiers du coffre d'or""",Le jeu aventure « Les templiers du coffre d’or...,Jeu,Famille,Détente,www.tourisme-cambresis.fr


,Unnamed: 0,url,title,description,cat1,cat2,cat3,domain
0,0,https://www.tourisme-cambresis.fr/1-les-templi...,le templier du coffr,le jeu aventur le templier du coffr créé par l...,Jeu,Famille,Détente,www.tourisme-cambresis.fr


___
### 3.Vectorisation des données
___
vous devez ensuite convertir les données textuelles prétraitées en vecteurs de caractéristiques en utilisant par exemple TfidfVectorizer de Scikit-learn. Vous pouvez concaténer les vecteurs obtenus à partir des colonnes title, description et éventuellement domain.

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy

vectorizer = TfidfVectorizer()
X_title = vectorizer.fit_transform(df['title'])
X_description = vectorizer.transform(df['description'])
X_domain = vectorizer.transform(df['domain'])

# Concaténer les vecteurs
from sklearn.datasets import load_files
X = scipy.sparse.hstack([X_title, X_description, X_domain])

<391x2238 sparse matrix of type '<class 'numpy.float64'>'
	with 11518 stored elements in Compressed Sparse Row format>